# Cvičenie 6: Q-Learning a SARSA

Na dnešnom cvičení sa budeme zaoberať algoritmami typu temporal difference, a to konkrétne Q-učením a algoritmom SARSA. Algoritmy budeme aplikovať pri riešení už známeho prostredia Gridworld.

## Q-učenie

Q-učenie (*Q-Learning*), podobne ako Monte Carlo metódy, sa učí priamo z interakcií s prostredím, jeho priebeh teda závisí od priebehu konkrétnych epizód. Q-Learning zároveň nevyžaduje známy model prostredia, prechody medzi stavmi sa buď nasimulujú, alebo určia zo skutočnej interakcie agenta s prostredím. Algoritmus zvyčajne pracuje s $\varepsilon$-greedy politikou, kde sa s najväčšou pravdepodobnosťou vyberie akcia, ktorú agent vníma ako najlepšiu možnú, pritom ale existuje možnosť výberu náhodnej akcie. Q-learning je ale off-policy metóda: to znamená, že pri aktualizácii odhadu hodnôt dvojice stav-akcia sa neberie do úvahy práve platná politika agenta.

Pseudokód nájdete na obrázku nižšie, prediskutujte jeho kroky a význam premenných a parametrov, s ktorými algoritmus pracuje.

<img src="lab06/q-learning.jpg">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Následne algoritmus aplikujeme pri riešení už známeho príkladu gridworld:

<img src="lab06/gridworld_mdp.jpg" width="600">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v strede sveta. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra ukončí. Pre ostatné kroky dostane agent odmenu -1. Pre stav označený $s_{32}$ môže byť odmena 5 s pravdepodobnosťou *0.1*.

Väčšina sveta je úplne deterministická. Na pozícii $s_{12}$ fúka silný vietor, ktorý môže agenta posunúť na juh aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na juh je $0.4$. Ak agent vyberie pohyb na juh, určite sa tam dostane.

Discount factor $\gamma = 0.8$. Učiaci parameter $\alpha = 0.2$; $\varepsilon = 0.1$. Prvý odhad hodnoty pre každú dvojicu stav-akcia bude 0: $Q(s, a) = 0 \ \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$.

## Ukážka fungovania algoritmu

Priebeh algoritmu si ukážeme na nasledovných epizódach:

$s_{32}, W, -1, s_{31}, S, -1, s_{31}, E, -1, s_{32}, S, -1, s_{32}, N, -10, s_{22}$

$s_{23}, E, -1, s_{23}, W, -10, s_{22}$

$s_{32}, E, -1 s_{33}, S, -1, s_{33}, N, -1, s_{23}, N, 10, s_{13}$

### 1. epizóda

Prvú epizódu začneme zo stavu $s_{32}$, a keďže na začiatku sme inicializovali všetky hodnoty $Q = 0$, akcia sa vyberie náhodne. Predpokladajme, že sa náhodne vybrala akcia posunu na západ (*W*), po ktorej sa ocitneme v stave $s_{31}$ a obdržíme odmenu $-1$. Aktualizovanú hodnotu potom vypočítame nasledovne:

$Q(s_{32}, W) \leftarrow Q(s_{32}, W) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{31}, a) - Q(s_{32}, W)) = 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0) = -0.2$

Podobne postupujeme pri ďalšom kroku - $s_{31}, S, -1, s_{31}$:

$Q(s_{31}, S) \leftarrow Q(s_{31}, S) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{31}, a) - Q(s_{31}, S)) = 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0) = -0.2$

Keďže sme ostali v rovnakom stave, znova hľadáme najlepšiu možnú akciu (s pravdepodobnosťou $\varepsilon$) pre daný stav. Q-hodnoty pre stav $s_{31}$ sú nasledovné: ${0.0; 0.0; -0.2; 0.0}$, teda je jasné, že posun na juh sa nevyberie (máme s ním zlé skúsenosti). Ak sa vyberie akcia posunu na východ a dostaneme sa do stavu $s_{32}$, tak aktualizácia prebehne nasledovným spôsobom:

$Q(s_{31}, E) \leftarrow Q(s_{31}, E) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{32}, a) - Q(s_{31}, E)) = 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0) = -0.2$

Podobne postupujeme pri ďalšom kroku:

$Q(s_{32}, S) \leftarrow Q(s_{32}, S) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{32}, a) - Q(s_{32}, S)) = 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0) = -0.2$

Pri poslednom kroku spadneme do pasce, pričom vieme, že maximálna hodnota pre terminálny stav musí byť $0$:

$Q(s_{32}, N) \leftarrow Q(s_{32}, N) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{22}, a) - Q(s_{32}, N)) = 0.0 + 0.2 \cdot (-10 + 0.8 \cdot 0.0 - 0.0) = -2$

Po tejto epizóde bude naša tabuľka vyzerať nasledovne:

| stav |   N  |   E  |   S  |   W  |
|:----:|:----:|:----:|:----:|:----:|
| s11  |    0 |    0 |    0 |    0 |
| s12  |    0 |    0 |    0 |    0 |
| s13  |    0 |    0 |    0 |    0 |
| s21  |    0 |    0 |    0 |    0 |
| s22  |    0 |    0 |    0 |    0 |
| s23  |    0 |    0 |    0 |    0 |
| s31  |    0 | -0.2 | -0.2 |    0 |
| s32  |   -2 |    0 | -0.2 | -0.2 |
| s33  |    0 |    0 |    0 |    0 |

### 2. epizóda

Pri Q-learningu vykonáme aktualizáciu hodnôt pre každý krok, teda pre druhú epizódu platia rovnaké výpočty:

$Q(s_{23}, E) \leftarrow Q(s_{23}, E) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{23}, a) - Q(s_{23}, E))$

$Q(s_{23}, E) \leftarrow 0.0 + 0.2 \cdot (-1 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{23}, E) \leftarrow -0.2$

Pre druhý krok:

$Q(s_{23}, W) \leftarrow Q(s_{23}, W) + \alpha \cdot (r + \gamma \cdot \underset{a}{max}Q(s_{22}, a) - Q(s_{23}, W))$

$Q(s_{23}, W) \leftarrow 0.0 + 0.2 \cdot (-10 + 0.8 \cdot 0.0 - 0.0)$

$Q(s_{23}, W) \leftarrow -2$

A Q-tabuľka bude vyzerať takto:

| stav |   N  |   E  |   S  |   W  |
|:----:|:----:|:----:|:----:|:----:|
| s11  |    0 |    0 |    0 |    0 |
| s12  |    0 |    0 |    0 |    0 |
| s13  |    0 |    0 |    0 |    0 |
| s21  |    0 |    0 |    0 |    0 |
| s22  |    0 |    0 |    0 |    0 |
| s23  |    0 | -0.2 |    0 |   -2 |
| s31  |    0 | -0.2 | -0.2 |    0 |
| s32  |   -2 |    0 | -0.2 | -0.2 |
| s33  |    0 |    0 |    0 |    0 |

### 3. epizóda

Spracujte 3. ukážkovú epizódu a skontrolujte získané Q-hodnoty podľa tabuľky:

| stav |   N  |   E  |   S  |   W  |
|:----:|:----:|:----:|:----:|:----:|
| s11  |    0 |    0 |    0 |    0 |
| s12  |    0 |    0 |    0 |    0 |
| s13  |    0 |    0 |    0 |    0 |
| s21  |    0 |    0 |    0 |    0 |
| s22  |    0 |    0 |    0 |    0 |
| s23  |    2 | -0.2 |    0 |   -2 |
| s31  |    0 | -0.2 | -0.2 |    0 |
| s32  |   -2 | -0.2 | -0.2 | -0.2 |
| s33  | -0.2 |    0 | -0.2 |    0 |

Ďalší priebeh algoritmu si môžete vizualizovať pomocou [ukážkového kódu](lab06/qlearning.zip). Porovnajte vypočítané Q-hodnoty s riešením Bellmanových rovníc optimality.

## SARSA

On-policy variantom algoritmu Q-Learning je SARSA, ktorý oproti Q-Learningu berie do úvahy aj nasledujúcu akciu pri aktualizácii Q-hodnoty. Prediskutujte jednotlivé kroky algoritmu a porovnajte ich s krokmi algoritmu Q-Learning.

<img src="lab06/sarsa.jpg">

Následne si stiahnite [implementáciu algoritmu SARSA](lab06/sarsa.py), ktorá ale obsahuje rovnaký kód, ako Q-learning. Opravte kód tak, aby implementoval algoritmus SARSA.